## Core code of the image processing function

This code module provides a series of image processing algorithms, including Sobel edge detection, Canny edge detection, Marr-Hildreth edge detection, Hough Transform, Active Contour (Snakes), and Thresholding-based Region Growing segmentation. Users can call different algorithms as needed and pass the corresponding parameters when calling.

In [1]:
from PIL import Image, ImageOps, ImageDraw
import numpy as np
import cv2
from skimage.segmentation import active_contour
from skimage.filters import gaussian
import os

def sobel_detector(input_image, ksize=3):
    # Convert to grayscale
    gray_image = ImageOps.grayscale(input_image)
    
    image_array = np.array(gray_image)
    
    # Apply Sobel filter
    sobel_x = cv2.Sobel(image_array, cv2.CV_64F, 1, 0, ksize=ksize)
    sobel_y = cv2.Sobel(image_array, cv2.CV_64F, 0, 1, ksize=ksize)
    
    # Calculate magnitude of the gradient
    magnitude = np.hypot(sobel_x, sobel_y)
    magnitude = (magnitude / magnitude.max()) * 255
    magnitude = magnitude.astype(np.uint8)
    
    # Convert back to PIL image
    return Image.fromarray(magnitude)

def canny_detector(input_image, low_threshold=50, high_threshold=150):

    # Convert to grayscale
    image_array = np.array(input_image.convert('L'))
    
    # Apply Canny edge detector
    edges = cv2.Canny(image_array, low_threshold, high_threshold)
    
    # Convert back to PIL image
    return Image.fromarray(edges)

def marr_hildreth_detector(input_image, sigma=3):

    # Convert to grayscale
    image_array = np.array(input_image.convert('L'))
    
    # Apply Gaussian blur
    blurred_image = cv2.GaussianBlur(image_array, (0, 0), sigma)
    
    # Apply Laplacian filter
    laplacian = cv2.Laplacian(blurred_image, cv2.CV_64F)
    
    # Find zero crossings
    zero_crossings = np.zeros_like(laplacian)
    for i in range(1, laplacian.shape[0] - 1):
        for j in range(1, laplacian.shape[1] - 1):
            if laplacian[i, j] == 0:
                if (laplacian[i+1, j] < 0 and laplacian[i-1, j] > 0) or (laplacian[i+1, j] > 0 and laplacian[i-1, j] < 0) or \
                   (laplacian[i, j+1] < 0 and laplacian[i, j-1] > 0) or (laplacian[i, j+1] > 0 and laplacian[i, j-1] < 0):
                    zero_crossings[i, j] = 255
            elif laplacian[i, j] < 0:
                if (laplacian[i+1, j] > 0 or laplacian[i-1, j] > 0 or laplacian[i, j+1] > 0 or laplacian[i, j-1] > 0):
                    zero_crossings[i, j] = 255
            elif laplacian[i, j] > 0:
                if (laplacian[i+1, j] < 0 or laplacian[i-1, j] < 0 or laplacian[i, j+1] < 0 or laplacian[i, j-1] < 0):
                    zero_crossings[i, j] = 255

    # Convert back to PIL image
    zero_crossings = zero_crossings.astype(np.uint8)
    return Image.fromarray(zero_crossings)

def hough_transform(input_image, canny_low_threshold=50, canny_high_threshold=150, hough_threshold=100, max_lines=50):

    # Convert to grayscale
    image_array = np.array(input_image.convert('L'))
    
    # Apply Canny edge detector
    edges = cv2.Canny(image_array, canny_low_threshold, canny_high_threshold)
    
    # Apply Hough Line Transform
    lines = cv2.HoughLines(edges, 1, np.pi / 180, hough_threshold)
    
    # Draw lines on the original image
    output_image = input_image.convert("RGB")
    draw = ImageDraw.Draw(output_image)
    if lines is not None:
        for i, line in enumerate(lines[:max_lines]):  # Limit the number of lines drawn
            rho, theta = line[0]
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * (a))
            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * (a))
            draw.line((x1, y1, x2, y2), fill=(255, 0, 0), width=2)
    return output_image
def active_contour_tracing(input_image, init_points, alpha=0.5, beta=0.001, gamma=0.005, sigma=40.0):

    # Convert to grayscale and apply Gaussian smoothing
    image_array = np.array(input_image.convert('L'))
    image_array = gaussian(image_array, sigma=1.0)
    
    # Apply active contour (snake) algorithm
    snake = active_contour(image_array, init_points, alpha=alpha, beta=beta, gamma=gamma)
    
    # Draw the snake contour on the original image
    output_image = input_image.convert("RGB")
    draw = ImageDraw.Draw(output_image)
    for i in range(len(snake) - 1):
        draw.line((snake[i][1], snake[i][0], snake[i + 1][1], snake[i + 1][0]), fill=(255, 0, 0), width=2)
    draw.line((snake[-1][1], snake[-1][0], snake[0][1], snake[0][0]), fill=(255, 0, 0), width=2)
    return output_image

def threshold_and_region_growing(input_image, threshold_value=128, tolerance=10):

    # Convert to grayscale
    image_array = np.array(input_image.convert('L'))
    
    # Apply thresholding
    _, binary_image = cv2.threshold(image_array, threshold_value, 255, cv2.THRESH_BINARY)
    
    # Perform region growing using flood fill
    binary_image = binary_image.astype(np.uint8)
    seed_point = (binary_image.shape[0] // 2, binary_image.shape[1] // 2)
    filled_image = cv2.floodFill(binary_image, None, seed_point, 255, flags=cv2.FLOODFILL_FIXED_RANGE, loDiff=(tolerance,)*3, upDiff=(tolerance,)*3)[1]
    
    # Convert back to PIL image
    return Image.fromarray(filled_image)

def process_image(input_image, method='sobel', **kwargs):
    if method == 'sobel':
        return sobel_detector(input_image, **kwargs)
    elif method == 'canny':
        return canny_detector(input_image, **kwargs)
    elif method == 'marr_hildreth':
        return marr_hildreth_detector(input_image, **kwargs)
    elif method == 'hough':
        return hough_transform(input_image, **kwargs)
    elif method == 'active_contour':
        return active_contour_tracing(input_image, **kwargs)
    elif method == 'threshold_region':
        return threshold_and_region_growing(input_image, **kwargs)
    else:
        raise ValueError(f"Unsupported method: {method}")


input_image = Image.open("capybara.jpg")
method = 'active_contour'  
if method == 'active_contour':
    s = np.linspace(0, 2*np.pi, 473)
    r = 140 + 95*np.sin(s)
    c = 190 + 150*np.cos(s)
    init_points = np.array([r, c]).T
    output_image = process_image(input_image, method=method, init_points=init_points)
elif method == 'hough':
    output_image = process_image(input_image, method=method, canny_low_threshold=50, canny_high_threshold=150, hough_threshold=100, max_lines=50)
elif method == 'threshold_region':
    output_image = process_image(input_image, method=method, threshold_value=110, tolerance=128)
elif method == 'sobel':
    output_image = process_image(input_image, method=method, ksize=3)
elif method == 'canny':
    output_image = process_image(input_image, method=method, low_threshold=100, high_threshold=300)
elif method == 'marr_hildreth':
    output_image = process_image(input_image, method=method, sigma=3.0)
else:
    output_image = process_image(input_image, method=method)
output_image.save("output_image.jpg")
output_image.show()


## Evaluating
For evaluating the edge detection and segmentation algorithms provided in the code, the Berkeley Segmentation Dataset and Benchmark (BSDS500) is a suitable choice. This dataset includes a variety of natural images along with human-annotated ground truth segmentations and boundary annotations, making it ideal for assessing the performance of different image processing algorithms.

## Why BSDS300?
Variety of Images: It includes a diverse set of natural images.
Human-Annotated Ground Truth: Provides multiple ground truth segmentations for each image, allowing for comprehensive evaluation.
Standard Benchmark: Widely used in the research community, ensuring that your results can be compared with existing literature.

## Evaluation: By using BSDS300 dataset

Outputs: Precision, recall, F1 score, Processing Time

In [ ]:
import os
import time
import numpy as np
import cv2
from PIL import Image, ImageOps, ImageDraw
from skimage.segmentation import active_contour
from skimage.filters import gaussian
import matplotlib.pyplot as plt

def sobel_detector(input_image, ksize=3):
    gray_image = input_image.convert('L')
    image_array = np.array(gray_image)
    sobel_x = cv2.Sobel(image_array, cv2.CV_64F, 1, 0, ksize=ksize)
    sobel_y = cv2.Sobel(image_array, cv2.CV_64F, 0, 1, ksize=ksize)
    magnitude = np.hypot(sobel_x, sobel_y)
    magnitude = (magnitude / magnitude.max()) * 255
    magnitude = magnitude.astype(np.uint8)
    return Image.fromarray(magnitude)

def canny_detector(input_image, low_threshold=50, high_threshold=150):
    image_array = np.array(input_image.convert('L'))
    edges = cv2.Canny(image_array, low_threshold, high_threshold)
    return Image.fromarray(edges)

def marr_hildreth_detector(input_image, sigma=1.0):
    image_array = np.array(input_image.convert('L'))
    blurred_image = cv2.GaussianBlur(image_array, (0, 0), sigma)
    laplacian = cv2.Laplacian(blurred_image, cv2.CV_64F)
    zero_crossings = np.zeros_like(laplacian)
    for i in range(1, laplacian.shape[0] - 1):
        for j in range(1, laplacian.shape[1] - 1):
            if laplacian[i, j] == 0:
                if (laplacian[i+1, j] < 0 and laplacian[i-1, j] > 0) or (laplacian[i+1, j] > 0 and laplacian[i-1, j] < 0) or \
                   (laplacian[i, j+1] < 0 and laplacian[i, j-1] > 0) or (laplacian[i, j+1] > 0 and laplacian[i, j-1] < 0):
                    zero_crossings[i, j] = 255
            elif laplacian[i, j] < 0:
                if (laplacian[i+1, j] > 0 or laplacian[i-1, j] > 0 or laplacian[i, j+1] > 0 or laplacian[i, j-1] > 0):
                    zero_crossings[i, j] = 255
            elif laplacian[i, j] > 0:
                if (laplacian[i+1, j] < 0 or laplacian[i-1, j] < 0 or laplacian[i, j+1] < 0 or laplacian[i, j-1] < 0):
                    zero_crossings[i, j] = 255
    zero_crossings = zero_crossings.astype(np.uint8)
    return Image.fromarray(zero_crossings)

def hough_transform(input_image, canny_low_threshold=50, canny_high_threshold=150, hough_threshold=100, max_lines=10):
    image_array = np.array(input_image.convert('L'))
    edges = cv2.Canny(image_array, canny_low_threshold, canny_high_threshold)
    lines = cv2.HoughLines(edges, 1, np.pi / 180, hough_threshold)
    output_image = input_image.convert("RGB")
    draw = ImageDraw.Draw(output_image)
    if lines is not None:
        for i, line in enumerate(lines[:max_lines]):
            rho, theta = line[0]
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * (a))
            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * (a))
            draw.line((x1, y1, x2, y2), fill=(255, 0, 0), width=2)
    return output_image

def active_contour_tracing(input_image, alpha=0.015, beta=10, gamma=0.001, sigma=1.0):
    image_array = np.array(input_image.convert('L'))
    image_array = gaussian(image_array, sigma=sigma)
    
    s = np.linspace(0, 2 * np.pi, 400)
    r = 100 + 50 * np.sin(s)
    c = 100 + 50 * np.cos(s)
    init_points = np.array([r, c]).T
    
    snake = active_contour(image_array, init_points, alpha=alpha, beta=beta, gamma=gamma)
    output_image = input_image.convert("RGB")
    draw = ImageDraw.Draw(output_image)
    for i in range(len(snake) - 1):
        draw.line((snake[i][1], snake[i][0], snake[i + 1][1], snake[i + 1][0]), fill=(255, 0, 0), width=2)
    draw.line((snake[-1][1], snake[-1][0], snake[0][1], snake[0][0]), fill=(255, 0, 0), width=2)
    return output_image

def threshold_and_region_growing(input_image, threshold_value=128, tolerance=10):
    image_array = np.array(input_image.convert('L'))
    _, binary_image = cv2.threshold(image_array, threshold_value, 255, cv2.THRESH_BINARY)
    binary_image = binary_image.astype(np.uint8)
    seed_point = (binary_image.shape[0] // 2, binary_image.shape[1] // 2)
    filled_image = cv2.floodFill(binary_image, None, seed_point, 255, flags=cv2.FLOODFILL_FIXED_RANGE, loDiff=(tolerance,)*3, upDiff=(tolerance,)*3)[1]
    return Image.fromarray(filled_image)

def process_image(input_image, method='sobel', **kwargs):
    if method == 'sobel':
        return sobel_detector(input_image, **kwargs)
    elif method == 'canny':
        return canny_detector(input_image, **kwargs)
    elif method == 'marr_hildreth':
        return marr_hildreth_detector(input_image, **kwargs)
    elif method == 'hough':
        return hough_transform(input_image, **kwargs)
    elif method == 'active_contour':
        return active_contour_tracing(input_image, **kwargs)
    elif method == 'threshold_region':
        return threshold_and_region_growing(input_image, **kwargs)
    else:
        raise ValueError(f"Unsupported method: {method}")

def evaluate_algorithm(gt_edges, pred_edges):
    precision = np.sum(pred_edges & gt_edges) / np.sum(pred_edges)
    recall = np.sum(pred_edges & gt_edges) / np.sum(gt_edges)
    f1_score = 2 * precision * recall / (precision + recall)
    return precision, recall, f1_score

def run_experiments_and_evaluate(image_paths, methods_params):
    results = []
    for image_path in image_paths:
        input_image = Image.open(image_path)
        gt_edges = np.array(input_image.convert('L')) > 128 

        for method, params_list in methods_params.items():
            for params in params_list:
                start_time = time.time()
                pred_image = process_image(input_image, method=method, **params)
                pred_edges = np.array(pred_image.convert('L')) > 128
                end_time = time.time()
                
                precision, recall, f1_score = evaluate_algorithm(gt_edges, pred_edges)
                processing_time = end_time - start_time
                results.append((image_path, method, params, precision, recall, f1_score, processing_time))
                print(f"Evaluated {method} with params {params} on {image_path}: Precision={precision}, Recall={recall}, F1={f1_score}, Time={processing_time:.2f}s")

    return results

def load_bsds300_images(dataset_path):
    image_paths = []
    for root, _, files in os.walk(dataset_path):
        for file in files:
            if file.endswith('.jpg'):
                image_paths.append(os.path.join(root, file))
    return image_paths

def main():
    dataset_path = 'BSDS300/images'  
    image_paths = load_bsds300_images(dataset_path)
    methods_params = {
        'sobel': [{'ksize': k} for k in [3, 5, 7]],
        'canny': [{'low_threshold': lt, 'high_threshold': ht} for lt in [50, 100] for ht in [150, 200]],
        'marr_hildreth': [{'sigma': s} for s in [0.5, 1.0, 2.0]],
        'hough': [{'canny_low_threshold': clt, 'canny_high_threshold': cht, 'hough_threshold': ht, 'max_lines': ml} 
                  for clt in [50, 100] for cht in [150, 200] for ht in [50, 100] for ml in [10, 20]],
        'active_contour': [{'alpha': a, 'beta': b, 'gamma': g, 'sigma': s} 
                           for a in [0.015, 0.02] for b in [10, 20] for g in [0.001, 0.01] for s in [1.0, 2.0]],
        'threshold_region': [{'threshold_value': tv, 'tolerance': t} for tv in [128, 150] for t in [10, 20]]
    }
    results = run_experiments_and_evaluate(image_paths, methods_params)
    
    results_file = 'experiment_results.txt'
    with open(results_file, 'w') as f:
        for result in results:
            f.write(f"{result}\n")
    print(f"Results saved to {results_file}")

if __name__ == "__main__":
    main()


## Results analyzation and plotting

In [ ]:
import ast
import pandas as pd
import matplotlib.pyplot as plt


def read_results(file_path):
    results = []
    with open(file_path, 'r') as f:
        for line in f:
            results.append(ast.literal_eval(line.strip()))
    return results

def analyze_results(results):
    df = pd.DataFrame(results, columns=['Image', 'Algorithm', 'Parameters', 'Precision', 'Recall', 'F1 Score', 'Processing Time'])
    
    mean_performance = df.groupby('Algorithm')[['Precision', 'Recall', 'F1 Score', 'Processing Time']].mean().reset_index()
    
    print("Mean Performance:")
    print(mean_performance)
    
    return df, mean_performance

def find_best_parameters(df):
    best_params = df.loc[df.groupby('Algorithm')['F1 Score'].idxmax()]
    print("\nBest Parameters for Each Algorithm:")
    print(best_params)
    return best_params

def plot_performance(mean_performance):
    fig, ax1 = plt.subplots(figsize=(12, 8))

    ax1.set_title('Performance Comparison of Different Algorithms')
    ax1.set_xlabel('Algorithm')
    ax1.set_ylabel('Score')
    ax1.bar(mean_performance['Algorithm'], mean_performance['Precision'], color='b', alpha=0.6, label='Precision')
    ax1.bar(mean_performance['Algorithm'], mean_performance['Recall'], color='g', alpha=0.6, label='Recall', bottom=mean_performance['Precision'])
    ax1.bar(mean_performance['Algorithm'], mean_performance['F1 Score'], color='r', alpha=0.6, label='F1 Score', bottom=mean_performance['Precision'] + mean_performance['Recall'])
    
    ax2 = ax1.twinx()
    ax2.set_ylabel('Processing Time (s)')
    ax2.plot(mean_performance['Algorithm'], mean_performance['Processing Time'], color='k', marker='o', linestyle='dashed', label='Processing Time')
    
    fig.tight_layout()
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')
    plt.xticks(rotation=45)
    plt.show()

def main():
    results_file = 'experiment_results.txt'
    results = read_results(results_file)
    
    df, mean_performance = analyze_results(results)
    best_params = find_best_parameters(df)
    
    plot_performance(mean_performance)

if __name__ == "__main__":
    main()
